In [1]:
import tensorflow as tf
import keras
import numpy as np 

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
import cv2
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array

In [3]:
# TRAIN_DIR = r'C:\Users\MCC\Desktop\landmark\trainset'
# train_folder_list=array(os.listdir(TRAIN_DIR))

# train_input = []
# train_label = []

# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(train_folder_list)
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# for index in range(len(train_folder_list)):
#     path = os.path.join(TRAIN_DIR, train_folder_list[index])
#     img_list = os.listdir(path)
#     for img in img_list:
#         img_path = os.path.join(path, img)
#         img = cv2.imread(img_path, cv2.IMREAD_COLOR)
#         train_input.append([np.array(img)])
#         train_label.append([np.array(onehot_encoded[index])])
        
# train_input = np.reshape(train_input, (-1,3518976))
# train_label = np.reshape(train_label, (-1,5))
# train_input = np.array(train_input).astype(np.float32)
# train_label = np.array(train_label).astype(np.float32)
# np.save("train_data.npy", train_input)
# np.save("train_label.npy", train_label)
train_input = np.load(r'C:\Users\MCC\Documents\jupyter\train_data.npy')
train_label = np.load(r'C:\Users\MCC\Documents\jupyter\train_label.npy')

In [4]:
# TEST_DIR = r'C:\Users\MCC\Desktop\landmark\testset'
# test_folder_list=array(os.listdir(TEST_DIR))

# test_input = []
# test_label = []

# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(test_folder_list)
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# for index in range(len(test_folder_list)):
#     path = os.path.join(TEST_DIR, test_folder_list[index])
#     img_list = os.listdir(path)
#     for img in img_list:
#         img_path = os.path.join(path, img)
#         img = cv2.imread(img_path, cv2.IMREAD_COLOR)
#         test_input.append([np.array(img)])
#         test_label.append([np.array(onehot_encoded[index])])
        
# test_input = np.reshape(test_input, (-1,3518976))
# test_label = np.reshape(test_label, (-1,5))
# test_input = np.array(test_input).astype(np.float32)
# test_label = np.array(test_label).astype(np.float32)
# np.save("test_data.npy", test_input)
# np.save("test_label.npy", test_label)
test_input = np.load(r'C:\Users\MCC\Documents\jupyter\test_data.npy')
test_label = np.load(r'C:\Users\MCC\Documents\jupyter\test_label.npy')

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 3518976])
y = tf.placeholder(tf.float32, shape=[None, 5])

x_image =tf.reshape(x, [-1, 1264, 928, 3])
x_test =tf.reshape(x, [-1, 1264, 928, 3])

trainphase = tf.placeholder(tf.bool)
keepprob = tf.placeholder(tf.float32)

In [6]:
def conv(x, in_ch, out_ch, name):
    with tf.variable_scope(name) as scope:
        w_conv =tf.get_variable(name='weights', shape=[3, 3, in_ch, out_ch], initializer=tf.contrib.layers.xavier_initializer_conv2d())
        h_bn = tf.layers.batch_normalization(tf.nn.conv2d(x, w_conv, strides=[1, 1, 1, 1], padding='SAME'), training=trainphase)
        h_conv = tf.nn.relu(h_bn)
    return h_conv

with tf.variable_scope("block1") as scope:
    h_conv1 = conv(x_image, 3, 2, "Conv1")
    h_conv2 = conv(h_conv1, 2, 2, "Conv2")
    h_conv2_pool = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.variable_scope("block2") as scope:
    h_conv3 = conv(h_conv2_pool, 2, 4, "Conv3")
    h_conv4 = conv(h_conv3, 4, 4, "Conv4")
    h_conv4_pool = tf.nn.max_pool(h_conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.variable_scope("block3") as scope:
    h_conv5 = conv(h_conv4_pool, 4, 8, "Conv5")
    h_conv6 = conv(h_conv5, 8, 8, "Conv6")
    h_conv6_pool = tf.nn.max_pool(h_conv6, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.variable_scope("block4") as scope:
    h_conv7 = conv(h_conv6_pool, 8, 16, "Conv7")
    h_conv8 = conv(h_conv7, 16, 16, "Conv8")
    h_conv8_pool = tf.nn.max_pool(h_conv8, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

with tf.variable_scope("block5") as scope:
    h_conv9 = conv(h_conv8_pool, 16, 32, "Conv9")
    h_conv10 = conv(h_conv9, 32, 32, "Conv10")
    h_conv10_pool = tf.nn.max_pool(h_conv10, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    

with tf.variable_scope("block7") as scope:
    h_conv11 = conv(h_conv10_pool, 32, 64, "Conv11")
    h_conv12 = conv(h_conv11, 64, 64, "Conv12")
    h_conv12_pool = tf.nn.max_pool(h_conv12, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        

with tf.variable_scope("block8") as scope:
    h_conv13 = conv(h_conv12_pool, 64, 128, "Conv13")
    h_conv14 = conv(h_conv13, 128, 128, "Conv14")
    h_conv14_pool = tf.nn.max_pool(h_conv14, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        
    
with tf.variable_scope("block9") as scope:
    h_conv15 = conv(h_conv14_pool, 128, 256, "Conv15")
    h_conv16 = conv(h_conv15, 256, 256, "Conv16")
    h_conv16_pool = tf.nn.max_pool(h_conv16, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        

with tf.variable_scope("block10") as scope:
    h_conv17 = conv(h_conv16_pool, 256, 512, "Conv17")
    h_conv18 = conv(h_conv17, 512, 512, "Conv18")
    h_conv18_pool = tf.nn.max_pool(h_conv18, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        
    
with tf.variable_scope("block11") as scope:
    h_conv19 = conv(h_conv18_pool, 512, 1024, "Conv19")
    h_conv20 = conv(h_conv19, 1024, 1024, "Conv20")
    h_conv20_pool = tf.nn.max_pool(h_conv20, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        

with tf.variable_scope("block12") as scope:
    h_conv21 = conv(h_conv20_pool, 1024, 2048, "Conv15")
    h_conv22 = conv(h_conv21, 2048, 2048, "Conv16")
    h_conv22_pool = tf.nn.max_pool(h_conv22, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')            
    
with tf.variable_scope("fclayer") as scope:
    h_conv22_pool_flat = tf.reshape(h_conv22_pool, [-1, 1*1*2048])
    h_conv22_pool_flat_dropout = tf.nn.dropout(h_conv22_pool_flat, keep_prob=keepprob)
    w_fc = tf.get_variable(name='weights', shape=[1*1*2048, 5], initializer=tf.contrib.layers.xavier_initializer())
    b_fc = tf.Variable(tf.constant(0.1, shape=[5]))
    logits = tf.matmul(h_conv22_pool_flat_dropout, w_fc) + b_fc
    y_pred = tf.nn.softmax(logits)
    
with tf.name_scope("l2_loss") as scope:
    vars = tf.trainable_variables()
    lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars]) * 0.0005
    
with tf.name_scope("trainer") as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)) + lossL2
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
    
with tf.name_scope("evaluation") as scope:
    correct_prediction = tf.equal(tf.argmax(y_pred, 1,), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_epoch = 1000
    for e in range(total_epoch):
        total_size = train_input.shape[0]
        batch_size = 50
        loss_list = []
        train_accuracy_list = []
        for i in range(int(total_size / batch_size)):
            batch_x = train_input[i*batch_size:(i+1)*batch_size]
            batch_y = train_label[i*batch_size:(i+1)*batch_size]
            
            sess.run(train_step, feed_dict={x: batch_x, y: batch_y, trainphase: True, keepprob: 0.7})
            
            train_accuracy = accuracy.eval(feed_dict={x: batch_x, y: batch_y, trainphase: True, keepprob: 1})
            loss_print = loss.eval(feed_dict={x: batch_x, y: batch_y, trainphase: False, keepprob: 1})
            train_accuracy_list.append(train_accuracy)
            loss_list.append(loss_print)
        print("반복(epcoh): ",e,"트레이닝 데이터 정확도: ",np.mean(train_accuracy_list), "손실함수: ", np.mean(loss_list))
        
        test_total_size = test_input.shape[0]
        test_batch_size = 5
    
        test_accuracy_list = []
        for i in range(int(test_total_size/ test_batch_size)):
            test_batch_x = test_input[i*batch_size:(i+1)*batch_size]
            test_batch_y = test_label[i*batch_size:(i+1)*batch_size]
        
            test_accuracy = accuracy.eval(feed_dict={x:test_batch_x, y: test_batch_y, trainphase: False, keepprob: 1})
            test_accuracy_list.append(test_accuracy)
        print("테스트 데이터 정확도: ", np.mean(test_accuracy_list))
        print()

반복(epcoh):  0 트레이닝 데이터 정확도:  0.5815 손실함수:  121013975000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  1 트레이닝 데이터 정확도:  0.075 손실함수:  203849520000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  2 트레이닝 데이터 정확도:  0.0 손실함수:  79159620000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  3 트레이닝 데이터 정확도:  0.0 손실함수:  113592305000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  4 트레이닝 데이터 정확도:  0.0 손실함수:  125641800000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  5 트레이닝 데이터 정확도:  0.0009999999 손실함수:  141025720000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  6 트레이닝 데이터 정확도:  0.007999999 손실함수:  118401800000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  7 트레이닝 데이터 정확도:  0.011499999 손실함수:  136149910000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  8 트레이닝 데이터 정확도:  0.024 손실함수:  173761480000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  9 트레이닝 데이터 정확도:  0.038 손실함수:  187655930000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  10 트레이닝 데이터 정확도:  0.042999998 손실함수:  171882610000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  11 트레이닝 데이터 정확도:  0.05 손실함수:  113812610000000.0
테스트 데이터 정확도:  nan

반복(epcoh):  12 트레이닝 데이터 정확도:  0.

In [ ]:
    test_total_size = test_input.shape[0]
    test_batch_size = 5
    
    test_accuracy_list = []
    for i in range(int(test_total_size/ test_batch_size)):
        test_batch_x = test_input[i*batch_size:(i+1)*batch_size]
        test_batch_y = test_label[i*batch_size:(i+1)*batch_size]
        
        test_accuracy = accuracy.eval(feed_dict={x:test_batch_x, y: test_batch_y, trainphase: False, keepprob: 1})
        test_accuracy_list.append(test_accuracy)
    print("테스트 데이터 정확도: ", np.mean(test_accuracy_list))
    print()